In [2]:
import os
import glob
import tomllib
import logging

from scipy.io import loadmat
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from hbmep.config import Config
from hbmep.model import BaseModel
from hbmep.model.utils import Site as Site

from hbmep_paper.utils import setup_logging
from models import RectifiedLogistic
from utils import read_data

logger = logging.getLogger(__name__)

TOML_PATH = "/home/vishu/repos/hbmep-paper/configs/tms/config.toml"
BUILD_DIR = "/home/vishu/repos/hbmep-paper/reports/foot/visit1/SCS08"
SUBDIR = "/home/vishu/data/raw/foot/visit1/SCS08"

FEATURES = ["participant"]
RESPONSE = ['PKPK_ADM', 'PKPK_APB', 'PKPK_Biceps', 'PKPK_ECR', 'PKPK_FCR', 'PKPK_Triceps']


config = Config(TOML_PATH)
config.BUILD_DIR = BUILD_DIR
base_model = BaseModel(config)
base_model._make_dir(BUILD_DIR)
setup_logging(
    dir=base_model.build_dir,
    fname="notebook"
)


2024-01-31 13:18:17,819 - hbmep_paper.utils.utils - INFO - Logging to /home/vishu/repos/hbmep-paper/reports/foot/visit1/SCS08/notebook.log


In [3]:
subdir = SUBDIR
df, mat = read_data(subdir)



In [14]:
config = Config(TOML_PATH)
config.BUILD_DIR = BUILD_DIR

model = RectifiedLogistic(config)
df, encoder_dict = model.load(df)
dest = os.path.join(model.build_dir, "mat.npy")
np.save(dest, mat)
model.plot(df=df, encoder_dict=encoder_dict, mep_matrix=mat)


2024-01-31 13:17:22,687 - hbmep.model.baseline - INFO - Initialized rectified_logistic
2024-01-31 13:17:22,687 - hbmep.dataset.core - INFO - Artefacts will be stored here - /home/vishu/repos/hbmep-paper/reports/foot/visit1/SCS08
2024-01-31 13:17:22,687 - hbmep.dataset.core - INFO - Copied config to /home/vishu/repos/hbmep-paper/reports/foot/visit1/SCS08
2024-01-31 13:17:22,688 - hbmep.dataset.core - WARNING - Total non-positive observations: 0
2024-01-31 13:17:22,689 - hbmep.dataset.core - WARNING - Total missing observations: 0
2024-01-31 13:17:22,689 - hbmep.dataset.core - INFO - Processing data ...
2024-01-31 13:17:22,690 - hbmep.utils.utils - INFO - func:load took: 0.00 sec
2024-01-31 13:17:22,696 - hbmep.model.baseline - INFO - Rendering ...
2024-01-31 13:17:23,531 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/foot/visit1/SCS08/dataset.pdf
2024-01-31 13:17:23,532 - hbmep.utils.utils - INFO - func:plot took: 0.84 sec


In [15]:
mcmc, posterior_samples = model.run_inference(df=df)


2024-01-31 13:17:27,762 - hbmep.model.baseline - INFO - Running inference with rectified_logistic ...


  0%|          | 0/6000 [00:00<?, ?it/s]

  0%|          | 0/6000 [00:00<?, ?it/s]

  0%|          | 0/6000 [00:00<?, ?it/s]

  0%|          | 0/6000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:



logger.info(mcmc.print_summary())
prediction_df = model.make_prediction_dataset(df=df)
posterior_predictive = model.predict(
    df=prediction_df,
    posterior_samples=posterior_samples
)
model.render_recruitment_curves(
    df=df,
    encoder_dict=encoder_dict,
    posterior_samples=posterior_samples,
    prediction_df=prediction_df,
    posterior_predictive=posterior_predictive
)
return

